## Load RMI Utilities Transition Hub Data (from https://utilitytransitionhub.rmi.org/data-download/ for Data Vault Prototype)

Copyright (C) 2023 OS-Climate

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

### We have local copies of the released datasets rooted in the S3_BUCKET : RMI/RMI-202*

### The next step is to enrich with OS-C Factor metadata

Contributed by Michael Tiemann (Github: MichaelTiemannOSC)

Load Credentials

In [1]:
# From the AWS Account page, copy the export scripts from the appropriate role using the "Command Line or Programmatic Access" link
# Paste the copied text into ~/credentials.env

import os
import io
from pathlib import Path
import shutil

import osc_ingest_trino as osc

import boto3
import zipfile

# import pyarrow as pa
# import pyarrow.parquet as pq
import json
import datetime

import numpy as np
import pandas as pd

In [2]:
# Load environment variables from credentials.env
osc.load_credentials_dotenv()

iceberg_catalog = "osc_datacommons_dev"
iceberg_schema = "rmi"
rmi_table_prefix = ""

engine = osc.attach_trino_engine(
    verbose=True, catalog=iceberg_catalog, schema=iceberg_schema
)
cxn = engine.connect()

using connect string: trino://MichaelTiemannOSC@trino-secure-odh-trino.apps.odh-cl2.apps.os-climate.org:443/osc_datacommons_dev/rmi


Create an S3 resource for the bucket holding source data

In [3]:
s3_source = boto3.resource(
    service_name="s3",
    endpoint_url=os.environ["S3_LANDING_ENDPOINT"],
    aws_access_key_id=os.environ["S3_LANDING_ACCESS_KEY"],
    aws_secret_access_key=os.environ["S3_LANDING_SECRET_KEY"],
)
source_bucket = s3_source.Bucket(os.environ["S3_LANDING_BUCKET"])

In [4]:
# bucket must be configured with credentials for trino, and accessible to the hive catalog
# You may need to use a different prefix here depending on how you name your credentials.env variables
hive_bucket = osc.attach_s3_bucket("S3_HIVE")

hive_catalog = "osc_datacommons_hive_ingest"
hive_schema = "ingest"

In [5]:
schema_create = osc._do_sql(
    f"""
create schema if not exists osc_datacommons_dev.{iceberg_schema}
WITH (
    location = 's3a://osc-datacommons-s3-bucket-dev02/data/{iceberg_schema}.db/'
)
""",
    engine,
    True,
)


create schema if not exists osc_datacommons_dev.rmi
WITH (
    location = 's3a://osc-datacommons-s3-bucket-dev02/data/rmi.db/'
)



Load RMI data file using pandas *read_csv* and appropriate dtype dictionaries

In [6]:
# Current (November 20, 2021) dataset sometimes represents YEAR as floating point number.  We will read as string and fix later.

dtype_dict = {
    # Table Name maps to dtypes (note that NaNs cannot encode to integers, so must do ex post facto fixes)
    "assets_earnings_investments": {
        "respondent_id": "int32",
        "asset_value": "float64",
        "earnings_value": "float64",
        "investment_value": "float64",
    },
    "customers_sales": {
        "respondent_id": "int32",
        "customers": "Int32",
        "sales": "float64",
        "revenues": "float64",
    },
    "debt_equity_returns": {
        "respondent_id": "int32",
        "rate_base_actual": "float64",
        "equity_actual": "float64",
        "debt_actual": "float64",
        "equity_ratio_actual": "float64",
        "returns_actual": "float64",
        "earnings_actual": "float64",
        "interest_actual": "float64",
        "fed_tax_expense_actual": "float64",
        "pre_tax_net_income_actual": "float64",
        "ROR_actual": "float64",
        "ROE_actual": "float64",
        "equity_ratio": "float64",
        "ROR": "float64",
        "ROE": "float64",
        "interest_rate": "float64",
        "effective_fed_tax_rate": "float64",
        "equity_authorized": "float64",
        "debt_authorized": "float64",
        "returns_authorized": "float64",
        "earnings_authorized": "float64",
        "interest_authorized": "float64",
        "interest_rate_authorized": "float64",
    },
    "emissions_targets": {
        "respondent_id": "Int32",
        "CO2_historical": "float64",
        "CO2_target": "float64",
        "CO2_target_all_years": "float64",
        "CO2_1point5C": "float64",
        "generation_historical": "float64",
        "generation_projected": "float64",
        "generation_1point5C": "float64",
        "CO2_intensity_historical": "float64",
        "CO2_intensity_target": "float64",
        "CO2_intensity_target_all_years": "float64",
        "CO2_intensity_1point5C": "float64",
    },
    "employees": {"respondent_id": "int32", "employees": "int32"},
    "expenditure_bills_burden": {
        "respondent_id": "int32",
        "expenditure": "float64",
        "bill": "float64",
        "burden": "float64",
    },
    "expenditure_bills_burden_detail": "string",
    "housing_units_income": {
        "respondent_id": "int32",
        "housing_units": "float64",
        "income": "float64",
    },
    "net_plant_balance": {
        "respondent_id": "int32",
        "original_cost": "float64",
        "accum_depr": "float64",
        "net_plant_balance": "float64",
        "ARC": "float64",
        "ARC_accum_depr": "float64",
        "net_ARC": "float64",
    },
    "operations_emissions_by_fuel": {
        "respondent_id": "int32",
        "plant_id_eia": "Int32",
        "latitude": "float64",
        "longitude": "float64",
        "operating_month": "Int32",
        "operating_year": "Int32",
        "retirement_month": "Int32",
        "retirement_year": "Int32",
        "generation": "float64",
        "fuel_consumption": "float64",
        "emissions_c02": "float64",
        "emissions_nox": "float64",
        "emissions_sox": "float64",
        "balancing_authority_name_eia": "object",
        "iso_rto_code": "object",
    },
    "operations_emissions_by_tech": {
        "respondent_id": "int32",
        "plant_id_eia": "Int32",
        "latitude": "float64",
        "longitude": "float64",
        "capacity": "float64",
        "year_end_capacity": "float64",
        "generation": "float64",
        "potential_generation": "float64",
        "capacity_factor": "float64",
        "fuel_consumption": "float64",
        "emissions_co2": "float64",
        "emissions_nox": "float64",
        "emissions_sox": "float64",
        "balancing_authority_name_eia": "object",
        "iso_rto_code": "object",
    },
    "revenue_by_tech": {
        "respondent_id": "int32",
        "revenue_total": "float64",
        "revenue_residential": "float64",
    },
    "state_policies": "string",
    "state_targets": "string",
    "utility_information": {
        "respondent_id": "int32",
        "utility_id_eia": "Int32",
        "duplicate_utility_id_eia": "boolean",
    },
    "utility_information_2023": {
        "respondent_id": "int32",
        "utility_id_ferc1": "Int32",
        "utility_id_ferc1_dbf": "Int32",
        "utility_id_eia": "Int32",
        "fraction_owned_utility": "float64",
        "duplicate_utility_id_eia": "boolean",
    },
    "utility_state_map": {
        "respondent_id": "int32",
        "capacity_owned_in_state": "float64",
        "capacity_operated_in_state": "float64",
        "mwh_sales_in_state": "float64",
    },
    "utility_state_map_2023": {
        "utility_id_eia": "Int32",
        "capacity_owned_in_state": "float64",
        "capacity_operated_in_state": "float64",
        "mwh_sales_in_state": "float64",
    },
}

fillna_dict = {
    "assets_earnings_investments": {
        "asset_value": 0,
        "earnings_value": 0,
        "investment_value": 0,
    },
    "customer_sales": {"customers": 0, "sales": 0, "revenues": 0},
}

dropna_dict = {
    "emissions_targets": {"respondent_id": "int32"},
}

Create the actual metadata for the source.  In this case, it is *rmi_utility_transition_hub*

We read and interpret the data dictionary that comes with the data sources.

In [7]:
if False:
    rmi_20210929_b = io.BytesIO(
        source_bucket.Object("RMI/RMI-20210929.zip").get()["Body"].read()
    )
    rmi_20210929_zip = zipfile.ZipFile(rmi_20210929_b, mode="r")
    del rmi_20210929_b
    # display(zipfile.ZipFile(rmi_20210929_zip, mode='r').filelist)
    rmi_dd = rmi_20210929_zip.read(
        "data_download/RMI Utility Transition Hub Data Dictionary.xlsx"
    )

    # Read all the sheets.  rmi_excel['sheet_name'] gives a specific sheet
    rmi_20210929_xls = pd.read_excel(rmi_dd, sheet_name=None, dtype=str)

    rmi_20211120_b = io.BytesIO(
        source_bucket.Object("RMI/RMI-20211120.zip").get()["Body"].read()
    )
    rmi_20211120_zip = zipfile.ZipFile(rmi_20211120_b, mode="r")
    del rmi_20211120_b
    # display(zipfile.ZipFile(rmi_20211120_zip, mode='r').filelist)
    rmi_dd = rmi_20211120_zip.read("data_dictionary.xlsx")

    # Read all the sheets.  rmi_excel['sheet_name'] gives a specific sheet
    rmi_20211120_xls = pd.read_excel(rmi_dd, sheet_name=None, dtype=str)

# When updated, we'll get the data dictionary from 2023 zipfile.
rmi_20220307_b = io.BytesIO(
    source_bucket.Object("RMI/RMI-20220307.zip").get()["Body"].read()
)
rmi_20220307_zip = zipfile.ZipFile(rmi_20220307_b, mode="r")
del rmi_20220307_b
# display(zipfile.ZipFile(rmi_20220307_zip, mode='r').filelist)
rmi_dd = rmi_20220307_zip.read("data_dictionary.xlsx")

# Read all the sheets.  rmi_excel['sheet_name'] gives a specific sheet
rmi_20220307_xls = pd.read_excel(io.BytesIO(rmi_dd), sheet_name=None, dtype=str)

del rmi_dd

rmi_20230202_b = io.BytesIO(
    source_bucket.Object("RMI/RMI-20230202.zip").get()["Body"].read()
)
rmi_20230202_zip = zipfile.ZipFile(rmi_20230202_b, mode="r")
del rmi_20230202_b

In [8]:
units_dict = {
    "$": "USD",
    "number": "",
}


def drop_cols_by_index(df, index):
    column_numbers = [
        x - 1 for x in range(df.shape[1], 0, -1)
    ]  # reversed list of columns' integer indices
    if type(index) == list:
        index.sort()
    else:
        index = [index]
    for i in index:
        # removing small-to-large index values from large-to-small list means we take from the right, preserving order from zero origin
        column_numbers.remove(i)
    column_numbers.sort()
    return df.iloc[:, column_numbers]  # return remaining columns


def generate_overview_meta(dd, release_date):
    global overview_dd
    print("Generating Overview...")
    title = dd.iloc[0, 0]
    general_overview = dd.iloc[0, 1]
    scope = dd.iloc[1, 1]
    limitations_to_scope = dd.iloc[2, 1]
    # Drop non-table data captured above
    dd = dd.iloc[4:].set_axis(["Name", "Description"], axis=1)
    meta_content = {
        "title": title,
        "description": general_overview,
        "version": f"Released {release_date}",
        "uri": "https://utilitytransitionhub.rmi.org/data-download/",
        "copyright": "© 2021 RMI",
        "license": "Creative Commons Attribution-Noncommercial 4.0 International Public License (CC BY-NC)",
        "contact": "utilitytransitionhub@rmi.org",
        "abstract": "\n".join([scope, limitations_to_scope]),
        "name": "rmi_utility_transition_hub",
    }
    overview_dd = dd
    return meta_content


def get_meta_fields_from_dd(dd):
    meta_fields = {
        k: {"description": v} for k, v in list(zip(dd["Data field"], dd["Definition"]))
    }
    for field, dim in list(zip(dd["Data field"], dd["Units"])):
        if dim == "":
            continue
        if dim in units_dict:
            meta_fields[field]["dimension"] = units_dict[dim]
        else:
            meta_fields[field]["dimension"] = dim
    return meta_fields

In [9]:
def generate_assets_meta(sheet, dd):
    global overview_dd
    description = overview_dd.loc[overview_dd["Name"] == sheet, "Description"].values[0]
    meta_content = {
        "tname": sheet,
        "parent_schema": schemaname,
        "description": description,
    }
    if dd.iloc[-1]["Data field"] == "Additional notes":
        meta_content["Additional_notes"] = "null"
        dd = dd.drop(dd.index[-1:])
    else:
        meta_content["Additional_notes"] = dd.iloc[-1]["Data field"]
        dd = dd.drop(dd.index[-2:])

    ### ??? What should we do with enumerations of asset and sub_asset types?

    # Drop columns that exist only to hold enumeration values
    dd = drop_cols_by_index(dd, [1, 2])
    # Asset & sub_asset are actually two fields
    fixup_index = dd[dd["Data field"].eq("asset & sub_asset")].index
    dd.loc[fixup_index, "Data field"] = "asset"
    dd.loc[fixup_index, "Definition"] = (
        "RMI's categorization of assets based on grouping of [steam,nuclear,hydro,renewables,other_fossil,transmission,distribution,other]"
    )
    dd.loc[fixup_index + 1, "Data field"] = "sub_asset"
    dd.loc[fixup_index + 1, "Definition"] = (
        "RMI's categorization of sub_assets when asset=other based on grouping of [AROs,construction_work_in_progress,distribution_arc,electric_plant_held_for_future_use,electric_plant_leased_to_others,experimental_plant,general_plant,general_plant_arc,hydro_arc,intangible_plant,net_ADIT,net_regulatory_assets,net_working_capital,nuclear_arc,other_deferred_debits_and_credits,other_electric_plant,other_fossil_arc,other_noncurrent_liabilities,regional_transmission_and_market_operation,renewables_arc,steam_arc,transmission_arc]"
    )
    dd = dd.dropna(subset=["Data field"]).fillna(value="")

    return get_meta_fields_from_dd(dd), meta_content

In [10]:
def generate_bills_meta(sheet, dd):
    global overview_dd
    description = overview_dd.loc[overview_dd["Name"] == sheet, "Description"].values[0]
    meta_content = {
        "tname": sheet,
        "parent_schema": schemaname,
        "description": description,
    }
    # Temporary fix for bad November 2020 data
    if sheet == "revenue_by_tech":
        dd = dd.rename(
            columns={
                " revenue_total ": "revenue_total",
                " revenue_residential ": "revenue_residential",
            }
        )
    if dd.iloc[-1]["Data field"] == "Additional notes":
        meta_content["Additional_notes"] = "null"
        dd = dd.drop(dd.index[-1:])
    else:
        meta_content["Additional_notes"] = dd.iloc[-1]["Data field"]
        dd = dd.drop(dd.index[-2:])

    ### ??? What should we do with code enumerations?

    # Drop columns that exist only to hold enumeration values
    dd = drop_cols_by_index(dd, 1)
    # Strip out enumerations of codes
    dd = dd.dropna(subset=["Data field"]).fillna(value="")
    return get_meta_fields_from_dd(dd), meta_content

In [11]:
def generate_emissions_meta(sheet, dd):
    global overview_dd
    global operations_emissions_fields, operations_emissions_content

    description = overview_dd.loc[overview_dd["Name"] == sheet, "Description"].values[0]
    meta_content = {
        "tname": sheet,
        "parent_schema": schemaname,
        "description": description,
    }
    if dd.iloc[-1]["Data field"] == "Additional notes":
        meta_content["Additional_notes"] = "null"
        dd = dd.drop(dd.index[-1:])
    else:
        meta_content["Additional_notes"] = dd.iloc[-1]["Data field"]
        dd = dd.drop(dd.index[-2:])

    ### Should we put code enumerations into dbt/seeds data files?

    # Drop columns that exist only to hold enumeration values
    dd = drop_cols_by_index(dd, 1)
    # Strip out enumerations of codes
    dd = dd.dropna(subset=["Data field"]).fillna(value="")
    dd[dd.columns[0]] = dd[dd.columns[0]].str.lower()
    meta_fields = get_meta_fields_from_dd(dd)
    meta_fields["utility_id_eia"] = {"description": "Utility Code from EIA"}
    del meta_fields["respondent_id"]
    meta_fields["utility_type_rmi"] = {
        "description": "Type of utility as classified by RMI"
    }
    meta_fields["operational_status_code"] = meta_fields["status"]
    del meta_fields["status"]
    if "fuel_type_code" in meta_fields:
        meta_fields["energy_source_code"] = meta_fields["fuel_type_code"]
        del meta_fields["fuel_type_code"]
    # Change fieldnames of quantified fields after interpreting their dimensionality
    meta_fields["net_generation"] = meta_fields["generation"]
    del meta_fields["generation"]
    meta_fields["fuel_consumed"] = meta_fields["fuel_consumption"]
    del meta_fields["fuel_consumption"]

    operations_emissions_fields = meta_fields
    operations_emissions_content = meta_content
    return meta_fields, meta_content

In [12]:
# This special case perfects the synthetic table we create ourselves (and completes the metadata describing it)


def generate_other_generation_meta(dd):
    global operations_emissions_fields
    meta_content = {
        "tname": "other_generation",
        "parent_schema": schemaname,
        "description": "EE & DR as well as Purchased Power, all of which count toward `avoided generation`",
        "Additional_notes": "Table derived from operation_emissions_by_field by OS-Climate",
    }
    meta_fields = operations_emissions_fields.copy()
    for field in list(meta_fields.keys()):
        if field not in dd.columns:
            print(f"Deleting obsolete field {field} from metadata")
            del meta_fields[field]
    return meta_fields, meta_content

In [13]:
def generate_generic_meta(sheet, dd):
    global overview_dd

    description = overview_dd.loc[overview_dd["Name"] == sheet, "Description"].values[0]
    meta_content = {
        "tname": sheet,
        "parent_schema": schemaname,
        "description": description,
    }
    if dd.iloc[-1]["Data field"] == "Additional notes":
        meta_content["Additional_notes"] = "null"
        dd = dd.drop(dd.index[-1:])
    else:
        meta_content["Additional_notes"] = dd.iloc[-1]["Data field"]
        dd = dd.drop(dd.index[-2:])
    return get_meta_fields_from_dd(dd), meta_content

The Data Dictionary is in an XLSX workbook
The actual Data lives in seprate CSV files

In [14]:
other_generation_meta_fields = None
other_generation_meta_content = None
other_generation_df = None


def generate_sheet_meta(wb, sheet, release_date, df):
    """
    WB: The workbook defining the Data Dictionary
    SHEET: The Data Dictionary for the table whose name is SHEET
    RELEASE_DATE: The release date (version) of the Data Dictionary
    DF: If not None, the dataframe for which we are constructing the metadata
    """

    # For as-yet unexplained reasons, RMI renamed the file assets_earnings_investments.csv
    # without updating the name of the sheet in the spreadsheet.  We switch to the intended sheetname here
    if sheet == "assets_earnings":
        sheet = "assets_earnings_investments"

    if sheet[-5:] == "_2023":
        dd = wb[sheet[:-5]]
    else:
        dd = wb[sheet]
    # Remove empty column that appears in all of the spreadsheets
    dd.columns = dd.iloc[0].values
    dd = dd.iloc[1:, 1:]

    if sheet == "Overview":
        meta_content = generate_overview_meta(dd, release_date)
        print("Metadata Overview")
        print(meta_content)
        return {}, meta_content
    if sheet == "data_sources":
        return {}, {}

    # All other sheets have Data fields (some of which need downcasing)
    dd["Data field"] = dd["Data field"].map(
        lambda x: x if not isinstance(x, str) or x == "Additional notes" else x.lower()
    )
    if sheet == "assets_earnings_investments":
        return generate_assets_meta(sheet, dd)
    if sheet == "emissions_targets":
        meta_fields, meta_content = generate_generic_meta(sheet, dd)
        meta_fields["target_scope"] = {
            "description": "Scope 1 (own generation) or Scope 3 (purchased generation)"
        }
        return meta_fields, meta_content
    if sheet == "utility_information":
        meta_fields, meta_content = generate_generic_meta(sheet, dd)
        meta_fields["entity_id"] = {"description": "XBRL-coded Utility ID from FERC"}
        return meta_fields, meta_content
    if sheet == "utility_information_2023":
        meta_fields, meta_content = generate_generic_meta("utility_information", dd)
        meta_fields = meta_fields.copy()
        meta_fields["ticker"] = meta_fields["parent_ticker"]
        del meta_fields["parent_ticker"]
        meta_fields["isin"] = meta_fields["parent_isin"]
        del meta_fields["parent_isin"]
        for obsolete_key in ["respondent_id", "first_report_year", "last_report_year"]:
            del meta_fields[obsolete_key]
        return meta_fields, meta_content
    if sheet == "utility_state_map_2023":
        meta_fields, meta_content = generate_generic_meta("utility_state_map", dd)
        meta_fields = meta_fields.copy()
        meta_fields["utility_id_eia"] = meta_fields["respondent_id"]
        del meta_fields["respondent_id"]
        return meta_fields, meta_content
    if sheet in [
        "employees",
        "expenditure_bills_burden",
        "expenditure_bills_burden_detail",
        "revenue_by_tech",
    ]:
        # Both tables have the same essential shape
        return generate_bills_meta(sheet, dd)
    if sheet in ["operations_emissions_by_fuel", "operations_emissions_by_tech"]:
        meta_fields, meta_content = generate_emissions_meta(sheet, dd)
        return meta_fields, meta_content
    if sheet == "other_generation":
        return generate_other_generation_meta(dd)
    # The sheet 'state_utility_policies' was renamed to 'state_policies' both in the dictionary and the filename
    if sheet in ["state_policies", "state_utility_policies"]:
        ### df = pd.read_csv(source_bucket.Object(f).get()['Body'],
        ###                  dtype={'respondent_id':'int32'},parse_dates=['date_updated'],dayfirst=True)
        return generate_generic_meta(sheet, dd)
    return generate_generic_meta(sheet, dd)

In [15]:
rmi_ingest_schemas = {  # 'rmi_20210929': ('September 2021', rmi_20210929_xls, rmi_20210929_zip),
    # 'rmi_20211120': ('November 2021', rmi_20211120_xls, rmi_20211120_zip),
    # 'rmi_20220307: ('March 2022', rmi_20220307_xls, rmi_20220307_zip),
    "rmi": ("February 2023", rmi_20220307_xls, rmi_20230202_zip),
}


def first_valid_col_value(df, col):
    return df.loc[df[col].first_valid_index(), col]


dbt_dict = {}
dbt_dict["models"] = dbt_models = {}
repo_root = Path().resolve().parent
models_dir = repo_root.joinpath("dbt", "rmi_transform", "models")

shutil.rmtree(models_dir, ignore_errors=True)
os.mkdir(models_dir, mode=0o755)

In [16]:
custom_meta_content = custom_meta_fields = None

# There is no datafile behind the data dictionary.  Run this to prime overview_dd, which all other metadata-finding depends upon
for schemaname, (release_date, workbook, zipfile) in rmi_ingest_schemas.items():
    # The dataframe for Non-Null data
    df_nn = None
    overview_dd = None
    operations_emissions_content = None
    operations_emissions_fields = None

    overview_meta_fields, overview_meta_content = generate_sheet_meta(
        workbook, "Overview", release_date, None
    )

    for zipinfo in zipfile.infolist():
        fname = zipinfo.filename
        ftimestamp = datetime.datetime(*zipinfo.date_time)
        if fname[-4:] != ".csv":
            continue
        if fname.startswith("__MACOSX/._"):
            continue

        tablename = fname.split("/")[-1].split(".")[0]
        # For as-yet unexplained reasons, RMI renamed the file assets_earnings_investments.csv
        # without updating the name of the sheet in the spreadsheet.  We switch to the intended name here
        if tablename == "assets_earnings":
            tablename = "assets_earnings_investments"
        if tablename == "expenditure_bills_burden_detail":
            continue
        print(f"fname: '{fname}'; tablename = '{tablename}'")
        with zipfile.open(fname) as zf:
            if tablename == "state_utility_policies":
                df = pd.read_csv(
                    zf,
                    dtype={"respondent_id": "int32"},
                    parse_dates=["date_updated"],
                    dayfirst=True,
                    engine="c",
                )
            elif tablename.startswith("utility"):
                df = pd.read_csv(zf, dtype=dtype_dict[tablename], engine="c")
                if release_date[-4:] <= "2021" and tablename == "utility_information":
                    # Correct information for 'American Transmission Co LLC', which is owned by 'WEC Energy Group'
                    df.loc[
                        df.respondent_id == 275,
                        ["parent_name", "parent_ticker", "parent_ISIN", "parent_LEI"],
                    ] = df.loc[
                        df.respondent_id == 519,
                        ["parent_name", "parent_ticker", "parent_ISIN", "parent_LEI"],
                    ].values
                    df = df.rename(
                        columns={
                            "parent_ISIN": "parent_isin",
                            "parent_LEI": "parent_lei",
                        }
                    )
                if release_date[-4:] < "2023" and tablename == "utility_information":
                    # Correct several LEI errors and omissions
                    df.loc[
                        df.parent_name == "American Electric Power Co., Inc.",
                        "parent_lei",
                    ] = "1B4S6S7G0TW5EE83BO58"
                    df.loc[
                        df.parent_name == "American States Water Co.", "parent_isin"
                    ] = first_valid_col_value(
                        df[df.parent_name == "American States Water Co."], "parent_isin"
                    )
                    df.loc[
                        df.parent_name == "American States Water Co.", "parent_lei"
                    ] = "529900L26LIS2V8PWM23"
                    # Berkshire Hathaway is 5493000C01ZX7D35SD85 but we're really dealing with the energy company,
                    # which has the LEI 549300JD0S5IZJE9LY15.  But while the energy company has a detailed revenue breakdown,
                    # it doesn't have a proper market cap.
                    df.loc[
                        df.parent_name == "Berkshire Hathaway, Inc.", "parent_lei"
                    ] = "549300JD0S5IZJE9LY15"
                    df.loc[df.parent_name == "Citizens Energy Corp.", "parent_lei"] = (
                        "5493008ORX814MK1WM19"
                    )
                    df.loc[df.parent_name == "FirstEnergy Corp.", "parent_lei"] = (
                        "549300SVYJS666PQJH88"
                    )
                    df.loc[df.parent_name == "LS Power", "parent_lei"] = (
                        "549300Z88AAE0R1YHI77"
                    )
                    # df.loc[df.parent_name=='National Grid PLC', 'parent_lei'] = 'MOM4570XTJ5YYX7JKH83'
                    # df.loc[df.parent_name=='National Grid PLC', 'parent_isin'] = 'US6362744095'
                    df.loc[df.parent_name == "NextEra Energy, Inc.", "parent_lei"] = (
                        "UMI46YPGBLUE4VGNNT48"
                    )
                    # AEP owns OVEC
                    df.loc[
                        df.parent_name == "Ohio Valley Electric Corp.", "parent_lei"
                    ] = "1B4S6S7G0TW5EE83BO58"
                    df.loc[
                        df.parent_name == "Ohio Valley Electric Corp.", "parent_ticker"
                    ] = "AEP"
                    df.loc[
                        df.parent_name == "Ohio Valley Electric Corp.", "parent_isin"
                    ] = "US0255371017"
                    df.loc[df.parent_name == "PG&E Corp.", "parent_lei"] = (
                        "8YQ2GSDWYZXO2EDN3511"
                    )
                    df.loc[df.parent_name == "Sempra", "parent_isin"] = (
                        first_valid_col_value(
                            df[df.parent_name == "Sempra"], "parent_isin"
                        )
                    )
                    df.loc[df.parent_name == "Sempra", "parent_lei"] = (
                        "PBBKGKLRK5S5C0Y4T545"
                    )
                    df.loc[df.parent_name == "Unitil Corp.", "parent_lei"] = (
                        "549300EYGHO5EZE7RL80"
                    )
                    df.loc[df.parent_name == "Verso Corp.", "parent_lei"] = (
                        "549300FODXCTQ8DGT594"
                    )
                    df.loc[df.parent_name == "Verso Corp.", "parent_isin"] = (
                        "US92531L2079"
                    )
                if tablename.startswith("utility_information"):
                    if tablename.endswith("2023"):
                        cols = ("parent_lei", "isin")
                    else:
                        cols = ("parent_lei", "parent_isin")
                    idx = df.parent_name == "Constllation Enrgy Commodities"
                    df.loc[idx, "parent_name"] = "Constellation Energy Commodities"
                    df.loc[idx, "utility_name"] = "Constellation Energy Commodities"
                    df.loc[idx, cols] = ["549300F8Y20RYGNGV346", "US21037T1097"]
                    df.loc[df.parent_name == "E.ON", cols] = [
                        "Q9MAIUP40P25UFBFG033",
                        "DE000ENAG999",
                    ]
                    # Exelon Corp
                    df.loc[df.parent_name == "Exelon Energy Co", cols] = [
                        "5493006XZOVPY83VHP23",
                        "US30161N1019",
                    ]
                    # NextEra Corp
                    df.loc[df.parent_name == "FPL Energy", cols] = [
                        "549300B8P43NZZ2P4080",
                        "US65339F1012",
                    ]
                    # Ørsted A/S
                    df.loc[df.parent_name == "Orsted", cols] = [
                        "W9NG6WMZIYEU8VEDOG48",
                        "DK0060094928",
                    ]
                    df.loc[df.parent_name == "Reliant Energy Inc", cols] = [
                        "21TPXMRRHFKOBHDC8J74",
                        "US6293775085",
                    ]
            elif tablename == "state_targets":
                df = pd.read_csv(zf, dtype=dtype_dict[tablename], engine="c")
                df.year = df.year.fillna("-1").astype("string")
                if release_date[-4:] < "2023":
                    df.loc[df.year.isin(["Annual", "2005/1990"]), "year"] = "-1"
                df.year = df.year.astype("int32")
            else:
                df = pd.read_csv(
                    zf, dtype=dtype_dict[tablename], thousands=",", engine="c"
                )
                if "year" in df.columns:
                    df.year = df.year.fillna("-1").astype("int32")
                if tablename == "revenue_by_tech":
                    # The 2020 numbers come with extra spaces, comma separators, and sometimes parentheses instead of minus signs.
                    def cleanup_2020_numbers(s):
                        if type(s) == float:
                            return s
                        if s is None or s == "" or pd.isnull(s):
                            return "nan"
                        s = s.strip().replace(",", "")
                        if s[0] == "(":
                            s = s[1:-1]
                        elif s == "#NAME?":
                            s = "nan"
                        return s

                    df.rename(
                        columns={
                            " revenue_total ": "revenue_total",
                            " revenue_residential ": "revenue_residential",
                        },
                        inplace=True,
                    )
                    if release_date[-4:] < "2022":
                        df.revenue_total = df.revenue_total.map(
                            cleanup_2020_numbers
                        ).astype("float64")
                        df.revenue_residential = df.revenue_residential.map(
                            cleanup_2020_numbers
                        ).astype("float64")
                elif tablename == "debt_equity_returns":
                    df.loc[~df.ROR_actual.map(np.isfinite), "ROR_actual"] = 0.0
                    df.loc[~df.ROE_actual.map(np.isfinite), "ROE_actual"] = 0.0

            df = df.convert_dtypes(
                infer_objects=False,
                convert_string=True,
                convert_integer=False,
                convert_boolean=False,
                convert_floating=False,
            )
            # df.info(verbose=True)

        if tablename in dropna_dict:
            df.dropna(subset=list(dropna_dict[tablename].keys()), inplace=True)
        if tablename in fillna_dict:
            df.fillna(value=fillna_dict[tablename], inplace=True)

        custom_meta_fields, custom_meta_content = generate_sheet_meta(
            workbook, tablename, release_date, df
        )
        if release_date[-4:] < "2023" and tablename in [
            "operations_emissions_by_fuel",
            "operations_emissions_by_tech",
        ]:
            # Both tables duplicate the 'Purchased Power' and 'EE & DR' data.
            # We only need one copy, which we create as 'other_generation'
            if df_nn is None:
                df_anon = df.loc[df["plant_name_eia"].isna()].copy()
                # Drop many NULL columns we don't need
                df_anon.dropna(axis=1, how="all", inplace=True)
                other_generation_df = df_anon
                custom_gen_meta_fields, custom_gen_meta_content = (
                    generate_other_generation_meta(df_anon)
                )

                iceberg_table = f"{rmi_table_prefix}other_generation"
                dbt_models[iceberg_table] = dbt_table = {
                    "description": custom_meta_content["description"]
                }
                if custom_meta_fields:
                    dbt_table["columns"] = dbt_columns = {
                        name: {"description": custom_meta_fields[name]["description"]}
                        for name in custom_meta_fields
                    }
                    for name in custom_meta_fields:
                        if "tags" in custom_meta_fields[name]:
                            dbt_columns[name]["tags"] = custom_meta_fields[name]["tags"]

                drop_table = osc._do_sql(
                    f"drop view if exists {iceberg_schema}.{iceberg_table}",
                    engine,
                    verbose=False,
                )
                drop_table = osc._do_sql(
                    f"drop table if exists {iceberg_schema}.{iceberg_table}_source",
                    engine,
                    verbose=False,
                )
                osc.fast_pandas_ingest_via_hive(
                    df_anon,
                    engine,
                    iceberg_catalog,
                    iceberg_schema,
                    f"{iceberg_table}_source",
                    hive_bucket,
                    hive_catalog,
                    hive_schema,
                    partition_columns=["year"],
                    overwrite=True,
                    verbose=False,
                )
                with open(
                    models_dir.joinpath(f"{iceberg_table}.sql"), "w", encoding="utf-8"
                ) as f:
                    print(
                        "{{ config(materialized='view', view_security='invoker') }}"
                        + f"""
with source_data as (
    select {", ".join(df_anon.columns)}
    from {iceberg_catalog}.{iceberg_schema}.{iceberg_table}_source
)
select * from source_data""",
                        file=f,
                    )
                df_nn = df.loc[~df["operating_month"].isna()]
            df.dropna(subset=["plant_name_eia"], inplace=True)
            # For some reason, data before 2010 is sometimes not filled in.
            for index, row in df[df["operating_month"].isna()].iterrows():
                # df_nn is only computed once, from either 'operations_emissions_by_fuel' or 'operations_emissions_by_tech'
                df0 = df_nn.loc[
                    (df_nn["respondent_id"] == row["respondent_id"])
                    & (df_nn["generator_id"] == row["generator_id"]),
                    ["operating_month", "operating_year"],
                ]
                if len(df0) == 0:
                    # In this case we have no prior data to refer to
                    continue
                om, oy = df0.iloc[0]
                df.loc[index, ("operating_month", "operating_year")] = om, oy
            # ICEBERG does not support integers less than 32 bits
            # for colname in ['operating_month', 'operating_year', 'retirement_month', 'retirement_year']:
            #     df[colname] = pd.to_numeric(df[colname],downcast='integer')
            #     pass
        elif tablename == "emissions_targets":
            # Needed because respondent_id 191 data is duplicated (and 121 is kinda duplicated, too).
            df.drop_duplicates(
                subset=None, keep="first", inplace=True, ignore_index=True
            )
            df.loc[
                df["CO2_intensity_historical"] == np.inf,
                ["CO2_historical", "CO2_intensity_historical"],
            ] = [0, 0]
            df["CO2_intensity_historical"] = df["CO2_intensity_historical"].astype(
                "float64"
            )

            # Fix discrepencies between emissions_targets and the other files that define/reference parent_name
            if release_date[-4:] < "2023":
                df.loc[df.parent_name == "American States Water", "parent_name"] = (
                    "American States Water Co."
                )
                df.loc[df.parent_name == "CMS Energy", "parent_name"] = (
                    "CMS Energy Corp."
                )
                df.loc[df.parent_name == "Emera Inc.", "parent_name"] = "Versant Power"
                df.loc[df.parent_name == "Fortis, Inc", "parent_name"] = "Fortis, Inc."
                df.loc[df.parent_name == "National Grid plc", "parent_name"] = (
                    "National Grid PLC"
                )
                df.loc[df.parent_name == "NorthWestern Corp.", "parent_name"] = (
                    "Northwestern Corp."
                )
                df.loc[df.parent_name == "OG&E Energy", "parent_name"] = (
                    "OG&E Energy Corp."
                )
                df.loc[df.parent_name == "PPL", "parent_name"] = "PPL Corp."
                df.loc[df.parent_name == "Sempra Energy", "parent_name"] = "Sempra"
                df.loc[df.parent_name == "Verso", "parent_name"] = "Verso Corp."
        # if tablename in tidy_dict:
        #     tidy_df = df.melt(id_vars=tidy_dict[tablename][0], value_vars=tidy_dict[tablename][2],
        #                       var_name=tidy_dict[tablename][1][0], value_name=tidy_dict[tablename][1][1])
        #     tidy_df[tidy_dict[tablename][1][0]] = tidy_df[tidy_dict[tablename][1][0]].apply(lambda x: x.split('_')[0])
        #     tidy_df.dropna(subset=[tidy_dict[tablename][1][1]],inplace=True)

        iceberg_table = f"{rmi_table_prefix}{tablename}"
        if custom_meta_content:
            dbt_models[iceberg_table] = dbt_table = {
                "description": custom_meta_content["description"]
            }
            dbt_table["meta"] = custom_meta_content.copy()
            del dbt_table["meta"]["description"]
            if custom_meta_fields:
                dbt_table["columns"] = dbt_columns = {
                    name: {"description": custom_meta_fields[name]["description"]}
                    for name in custom_meta_fields
                }
                for name in custom_meta_fields:
                    if "tags" in custom_meta_fields[name]:
                        dbt_columns[name]["tags"] = custom_meta_fields[name]["tags"]
        elif custom_meta_fields:
            raise VALUE_ERROR

        if "year" in df.columns:
            partition = "year"
        elif "first_report_year" in df.columns:
            partition = "first_report_year"
        elif "state" in df.columns:
            partition = "state"
        elif "utility_type_rmi" in df.columns:
            partition = "utility_type_rmi"
        else:
            raise KeyError
        drop_table = osc._do_sql(
            f"drop view if exists {iceberg_schema}.{iceberg_table}",
            engine,
            verbose=False,
        )
        drop_table = osc._do_sql(
            f"drop table if exists {iceberg_schema}.{iceberg_table}_source",
            engine,
            verbose=False,
        )
        if len(df) > 2000:
            print("hive->iceberg path...")
            osc.fast_pandas_ingest_via_hive(
                df,
                engine,
                iceberg_catalog,
                iceberg_schema,
                f"{iceberg_table}_source",
                hive_bucket,
                hive_catalog,
                hive_schema,
                partition_columns=[partition],
                overwrite=True,
                verbose=False,
            )
        else:
            print("trino ingest path...")
            columnschema = osc.create_table_schema_pairs(df)
            tabledef = f"""
create table if not exists {iceberg_catalog}.{iceberg_schema}.{iceberg_table}_source(
{columnschema}
) with (
    format = 'ORC',
    partitioning = array['{partition}']
)
"""
            qres = osc._do_sql(tabledef, engine, verbose=False)
            df.to_sql(
                f"{iceberg_table}_source",
                con=engine,
                schema=iceberg_schema,
                if_exists="append",
                index=False,
                method=osc.TrinoBatchInsert(batch_size=5000, verbose=False),
            )
        with open(
            models_dir.joinpath(f"{iceberg_table}.sql"), "w", encoding="utf-8"
        ) as f:
            print(
                "{{ config(materialized='view', view_security='invoker') }}"
                + f"""
with source_data as (
    select {", ".join(map(str.lower, df.columns))}
    from {iceberg_catalog}.{iceberg_schema}.{iceberg_table}_source
)
select * from source_data""",
                file=f,
            )
    zipfile.close()

print("Done!")

Generating Overview...
Metadata Overview
{'title': 'General Overview', 'description': 'The RMI Utility Transition Hub Data Download is a collection of publicly available data, organized and used to calculate key metrics that describe the US utility transition.\n\nThis data dictionary describes each data file in detail, including definitions, units, data sources, and methodology.\n\nFor downloadable data, visit https://utilitytransitionhub.rmi.org/data-download/.\nFor interactive data visualizations, visit https://utilitytransitionhub.rmi.org/portal/.\nFor analyses and insights, visit https://utilitytransitionhub.rmi.org/insights/.', 'version': 'Released February 2023', 'uri': 'https://utilitytransitionhub.rmi.org/data-download/', 'copyright': '© 2021 RMI', 'license': 'Creative Commons Attribution-Noncommercial 4.0 International Public License (CC BY-NC)', 'contact': 'utilitytransitionhub@rmi.org', 'abstract': 'Utilities coverage: all current FERC Form 1 respondents. This includes 403 t

Iterate through tablenames until we get to *Additional Information*, storing all the names and descriptions of the tables.

We've already collected the field names, types, dimensions, etc., of each table into our dataframes.
But there's additional metadata we can collect from the data dictionary

## Load metadata following an ingestion process into trino metadata store

### The schema is *metastore*, and the table names are *meta_schema*, *meta_table*, *meta_field*

In [17]:
dbt_yml = open(
    repo_root.joinpath("dbt", "rmi_transform", "rmi_base_schema.yml"),
    "w",
    encoding="utf-8",
)

In [18]:
print("version: 2", file=dbt_yml)


def json_dumps_no_lint(o, prefix_len, comma=False):
    s = json.dumps(o)
    if comma:
        s = s + ","
    if len(s) < (120 - prefix_len):
        return s
    return s + " # yamllint disable-line rule:line-length"


indent = 0
print("\nmodels:", file=dbt_yml)
indent = indent + 2
for name in dbt_dict["models"]:
    model = dbt_dict["models"][name]
    print(f"{' ' * indent}- name: {name}", file=dbt_yml)
    indent = indent + 2
    prefix = f"{' ' * indent}description:"
    print(prefix, json_dumps_no_lint(model["description"], len(prefix)), file=dbt_yml)
    meta = model.get("meta", None)
    if meta:
        print(f"{' ' * indent}config:", file=dbt_yml)
        indent = indent + 2
        print(f"{' ' * indent}meta: {{", file=dbt_yml)
        indent = indent + 2
        for meta_key in meta:
            prefix = f"{' ' * indent}{meta_key}:"
            print(
                prefix,
                json_dumps_no_lint(meta[meta_key], len(prefix), comma=True),
                file=dbt_yml,
            )
        indent = indent - 4
        print(f"{' ' * indent}  }}", end="", file=dbt_yml)
    print(f"\n{' ' * indent}columns:", file=dbt_yml)
    indent = indent + 2
    columns = model["columns"]
    for col in columns:
        print(f"{' ' * indent}- name: {col}", file=dbt_yml)
        indent = indent + 2
        for col_meta in columns[col]:
            prefix = f"{' ' * indent}{col_meta}:"
            print(
                prefix,
                json_dumps_no_lint(columns[col][col_meta], len(prefix)),
                file=dbt_yml,
            )
        indent = indent - 2
    indent = indent - 4
indent = indent - 2
assert indent == 0

In [19]:
dbt_yml.close()